# Check data in Teradata

## Connect to Teradata

In [5]:
import teradataml as tdml
from teradataml import DataFrame

In [2]:
import json

In [3]:
with open('teradata.json') as json_file:
    td_conf = json.load(json_file)

tdml.create_context(host=td_conf['host'],
                username=td_conf['user'],
                password=td_conf['password'],
                database=td_conf['schema'])
conn = tdml.get_connection()

/opt/conda/lib/python3.9/site-packages/teradataml/context/context.py:481: TeradataMlRuntimeWarning: Warning: Password is URL encoded.
  warnings.warn("Warning: Password is URL encoded.", category=TeradataMlRuntimeWarning)


## Show ingested data

In [12]:
df = DataFrame.from_query("select * from STG_IRIS_EV")
df.shape

(38, 5)

In [13]:
df.head()

float_input_0,float_input_1,float_input_2,float_input_3,ACQ_DTTM
4.8,3.4,1.6,0.2,2023-03-14 13:33:10
4.9,3.1,1.5,0.2,2023-03-14 13:33:10
5.0,2.0,3.5,1.0,2023-03-14 13:33:10
5.0,3.4,1.6,0.4,2023-03-14 13:33:10
5.0,3.0,1.6,0.2,2023-03-14 13:33:10
5.1,3.8,1.5,0.3,2023-03-14 13:33:10
5.0,2.3,3.3,1.0,2023-03-14 13:33:10
4.9,3.1,1.5,0.1,2023-03-14 13:33:10
4.8,3.0,1.4,0.3,2023-03-14 13:33:10
4.5,2.3,1.3,0.3,2023-03-14 13:33:10


## Score data with the logistic regression model

In [14]:
df = DataFrame.from_query(" \
SELECT * FROM mldb.onnxpredict( \
ON ( \
  select \
  cast(float_input_0 as decimal(18,2)) as float_input_0, \
  cast(float_input_1 as decimal(18,2)) as float_input_1, \
  cast(float_input_2 as decimal(18,2)) as float_input_2, \
  cast(float_input_3 as decimal(18,2)) as float_input_3 \
  from STG_IRIS_EV \
  ) AS InputTable \
ON (SELECT * FROM scikit_models WHERE model_id = 'logreg_iris_v1')  AS ModelTable  DIMENSION \
  USING \
    Accumulate ('*') \
) as whatever")
df.tail()

float_input_0,float_input_1,float_input_2,float_input_3,json_report
7.000,3.200,4.700,1.400,"{""output_probability"":[{""0"":0.0024769553,""1"":0.85516924,""2"":0.14235385}],""output_label"":[1]}"
6.700,3.300,5.700,2.500,"{""output_probability"":[{""0"":8.730148E-6,""1"":0.018769793,""2"":0.9812215}],""output_label"":[2]}"
6.700,3.000,5.200,2.300,"{""output_probability"":[{""0"":1.0419553E-4,""1"":0.09560589,""2"":0.9042899}],""output_label"":[2]}"
6.700,2.500,5.800,1.800,"{""output_probability"":[{""0"":1.2526436E-5,""1"":0.06381021,""2"":0.93617725}],""output_label"":[2]}"
6.500,2.800,4.600,1.500,"{""output_probability"":[{""0"":0.0033763452,""1"":0.78528553,""2"":0.21133809}],""output_label"":[1]}"
6.500,3.000,5.200,2.000,"{""output_probability"":[{""0"":2.3936074E-4,""1"":0.17774457,""2"":0.82201606}],""output_label"":[2]}"
6.500,3.000,5.500,1.800,"{""output_probability"":[{""0"":1.0249372E-4,""1"":0.14601514,""2"":0.8538824}],""output_label"":[2]}"
6.900,3.100,5.400,2.100,"{""output_probability"":[{""0"":7.0639755E-5,""1"":0.11276507,""2"":0.88716435}],""output_label"":[2]}"
7.700,3.800,6.700,2.200,"{""output_probability"":[{""0"":1.9986751E-7,""1"":0.006974007,""2"":0.9930258}],""output_label"":[2]}"
7.700,3.000,6.100,2.300,"{""output_probability"":[{""0"":9.3358443E-7,""1"":0.017022539,""2"":0.98297656}],""output_label"":[2]}"


**...in real use-case you would need to store results in some table**